In [11]:
### Import Libraries

from lightgbm import LGBMRegressor
import numpy as np 
import pandas as pd 
import os
from numpy import mean


In [12]:
!pip install flaml


In [13]:
!pip install scikit-learn==0.24

In [20]:

from flaml import AutoML


import warnings
warnings.filterwarnings('ignore')

In [15]:


train_path = '/kaggle/input/petfinder-pawpularity-score/train'
test_path = '/kaggle/input/petfinder-pawpularity-score/test'

df_train = pd.read_csv(train_path+'.csv')
df_test = pd.read_csv(test_path+'.csv')


In [16]:
# Take the train set columns as features

feature_cols=list(df_train.columns)[1:13]

# Values of train and test dataframes

train_set = np.array(df_train[feature_cols])
test_set = np.array(df_test[feature_cols])


## bin the pawpularity score among a list of values ranging from 0 to 100
### Labels is a list of pawpularity score values ranging from 1 to 10

df_train['label'] = pd.cut(np.array(df_train['Pawpularity']),
                          bins = list(range(0,110,10)),
                          labels = list(range(0,10)))


## Calculate the average pawpularity score of pets 
## The pet score can be between 1 to 10

paw_values = []
for i in range(0,10):
    v = list(df_train[df_train['label'] == i]['Pawpularity'])
    paw_values.append(mean(v))
ytrain = np.array(df_train['label'])

In [17]:
import flaml
from flaml import AutoML
import warnings
warnings.filterwarnings('ignore')

a = AutoML()
settings = {
    'estimator_list':['xgboost','rf','lgbm','catboost'],
    'log_file_name':'pp.log',
    'task':'regression',
    'metric':'rmse',
    'time_budget':360,
    'seed':2021
}
a.fit(train_set, ytrain,**settings)

In [ ]:

model = LGBMRegressor(colsample_bytree=0.9391605649208727,
               learning_rate=0.19814546649229234, max_bin=1023,
               min_child_samples=57, n_estimators=38, num_leaves=23,
               reg_alpha=0.0010079442745583322, reg_lambda=1.6292994966548278,
               verbose=-1)

model.fit(train_set, ytrain)
y_pred = a.predict(test_set)
rmse = mean_squared_error(y_pred, y_train, squared=False)
print(rmse)


In [ ]:
## Predicted output

pred = a.predict(test_set)

## Iterate through the paw values list and 
## update the final predicted pawpularity scores
pred_set = [paw_values[i] for i in pred]

## Convert the 
pred_df = pd.DataFrame.from_dict({
    "Id":list(df_test["Id"]),
    "Pawpularity":pred_set
})
pred_df.to_csv('./submission.csv',index=False)
pred_df.head(5)

In [ ]:
# Convert submission file to .csv and download the submission file

pred_df.to_csv('/kaggle/working/submission.csv')

from IPython.display import FileLink
FileLink(r'submission.csv')